<a href="https://colab.research.google.com/github/kayohr/valuation_stocks_fundamentus/blob/main/valuation_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install fundamentus

In [53]:
import fundamentus
import pandas as pd
import matplotlib.pyplot as plt

In [54]:
df = fundamentus.get_resultado()

In [ ]:
df.head()

Multiples,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,evebitda,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y
papel,,,,,,,,,,,,,,,,,,,,
AALR3,10.09,-5.24,1.30,1.012,0.0000,0.447,-5.68,30.05,-0.99,50.86,13.20,0.0337,-0.1853,0.0171,-0.2484,0.71,1522150.0,9.174560e+08,1.14,0.0351
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.1450,0.00,0.0,5.871730e+09,0.00,0.4586
ABCB4,22.83,6.35,0.92,0.000,0.0674,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.1450,0.00,14723400.0,5.871730e+09,0.00,0.4586
ABEV3,12.02,13.06,2.40,2.375,0.0607,1.428,-42.63,11.13,-11.88,10.38,7.67,0.2133,0.1876,0.1790,0.1836,0.89,380995000.0,7.896930e+10,0.04,0.1211
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,33.67,0.1029,-0.0096,0.0278,-0.0082,2.09,0.0,2.920600e+08,1.31,0.1641


In [ ]:
print(df.columns)

Index(['cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit', 'pacl',
       'evebit', 'evebitda', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc',
       'liq2m', 'patrliq', 'divbpatr', 'c5y'],
      dtype='object', name='Multiples')


In [ ]:
#Indicadores que não estão na biblioteca, caso precise usar.

ticker = df.loc['B3SA3'] #caso deseje usar esse ticker para as funções abaixo

#LPA
calc_lpa = ticker['cotacao'] / ticker['pl']
print("LPA: ", round(calc_lpa,2))

#Dividendo por ação
calc_dy_acao = ticker['dy'] * ticker['cotacao']
print("Dividendo por ação últimos 12 meses: ", round(calc_dy_acao, 2))

#Payout
calc_payout = calc_dy_acao / calc_lpa
print(f"Payout: { round(calc_payout, 2)*100}%")

LPA:  0.73
Dividendo por ação últimos 12 meses:  0.46
Payout: 62.0%


In [ ]:
df.loc[['BBAS3'],['cotacao']]

Multiples,cotacao
papel,
BBAS3,27.38


In [ ]:
df.loc['BBAS3']

In [ ]:
#Estimando o crescimento dos dividendos

def estimar_crescimento_dividendos(ativo):

    empresa = df.loc[ativo]

    #Função para achar a taxa de crescimento sustentavel maximo "g"
    roe = empresa['roe'] * 100
    payout = 92.28 / 100

    g1 = roe * (1 - payout)
    print(f"Taxa crescimento sustentável: {round(g1, 2)}%")

    #Lucro historico
    lucro1 =  4132512000  #lucro atual pegar no site
    lucro2 = 2713507000   #Lucro do ano a sua escolha no historico *pegar no site esse lucro. Irei opegar de 5 anos atrás
    n = 5 - 1
    g2 = ((lucro1 / lucro2) ** (1 / n) - 1) * 100

    print(f"A empresa cresceu: {round(g2, 2)}% nos últimos 5 anos")


estimar_crescimento_dividendos('B3SA3') #ticker

Taxa crescimento sustentável: 1.57%
A empresa cresceu: 11.09% nos últimos 5 anos


In [55]:
#Modelo de Gordon/Analise individual
#Esse modelo visa trazer o preço justo de uma empresa que ja alacaçou sua maturidade e basea sua projeção de crescimento contínuo dos dividendos pagos por uma empresa.

def modelo_gordon(ativo):

    empresa = df.loc[ativo]

    #Função para achar a taxa de crescimento "g" / Pegar Roe historico, pegar no site
    roe = 17.98 / 100
    payout = 92.28 / 100

    g = roe * (1 - payout)
    print(f"Taxa crescimento sustentável: {round(g, 2)} %")

    rf = (6.07 + 4.14) / 100   #Juros do IPC a longo prazo + inflação atual(IPCA anual)
    b = 0.54                   #Beta da empresa para analisar
    rm = (9.57) / 100          #Premio de Risco(Peguei da tabela do Damodaran de risco no Brasil)

    #Função para achar o ke(Custo de Capital Próprio)
    ke = rf + b * (rm - rf)
    print(f"Valor do Kr {round(ke, 2)} %")

    #Função para achar o valor justo
    d1 = 0.67  #Pode pegar o dividendo do ano annterior e multiplar pela a formula acima do Taxa crescimento sustentável: g1 = roe * (1 - payout)
    p = d1 / (ke - g)

    if empresa['cotacao'] < p:
        print(f"Preço de Gordon é: R${round(p, 2)} reais, e a ação hoje está abaixo do preço atual {empresa['cotacao']} ")
    else:
        print(f"Preço de Gordon é: R${round(p, 2)} reais, e a ação hoje está acima do preço atual de R${empresa['cotacao']} reais")


modelo_gordon('B3SA3') #ticker


Taxa crescimento sustentável: 0.01 %
Valor do Kr 0.1 %
Preço de Gordon é: R$7.9 reais, e a ação hoje está acima do preço atual de R$11.22 reais


In [ ]:
#Modelo de Gordon/Analise individual adaptado PL, multiplos justos

def multiplo_justo(ativo, roe, rtx):
    empresa = df.loc[ativo]

    roe = empresa[roe] * 100
    payout = 92.28 / 100
    r = rtx / 100                      #rtx(retorno da taxa exigida) pelo investidor. pode ser a tabela do Damodaran. Aqui vou usar o meu.

    g = ((1 - payout) * roe) / 100
    print(f"taxa de {round(g, 2) * 100}% ")

    pl = payout * (1 + g) / (r - g)
    print(f"PL justo. Fica barato abaixo de: {round(pl, 2)}")

    if empresa['pl'] >= pl:
        print(f"Caro. O PL é de {empresa['pl']}. Acima do justo...")
    else:
        print("Barato. Abaixo do justo!")


multiplo_justo('B3SA3', 'roe', 10) #ticker


taxa de 2.0% 
PL justo. Fica barato abaixo de: 11.12
Caro. O PL é de 14.8. Acima do justo...


In [ ]:
def pl_return(ativo, payout):

    pl_inverso = 1 / ativo['pl'] * (100)
    pl_inverso2 = pl_inverso * payout
    retorno_pl_inverso2 = (pl_inverso * ativo['pl']) / pl_inverso2

    resultado = f"""
  Price to Earnings do PL inverso do ativo {ativo.name} é {pl_inverso:.2f}%.
  E seu PL é {ativo['pl']}, então levaria {ativo['pl']} anos pagando 100% do seu lucro com DY de {pl_inverso:.2f}% ano para
  ter seu retorno total investido na ação.
  Porém, se ela não paga isso, se paga um payout de {round(payout, 2)*100}%  ela vai demorar {retorno_pl_inverso2:.2f} anos para devolver
  o capital com {pl_inverso2:.2f}% DY ao ano.
    """
    return resultado


# Calcula o valor do P/L inverso multiplicado pelo payout
print(pl_return(ticker, calc_payout))









  Price to Earnings do PL inverso do ativo B3SA3 é 6.76%.
  E seu PL é 14.8, então levaria 14.8 anos pagando 100% do seu lucro com DY de 6.76% ano para
  ter seu retorno total investido na ação.
  Porém, se ela não paga isso, se paga um payout de 62.0%  ela vai demorar 23.75 anos para devolver
  o capital com 4.21% DY ao ano.
    


In [ ]:
#Taxa de retorno liquido subtraindo PL do titulo publico para ver o spread
def pl_bonds(ativo):
  pl_inverso = 1 / ativo['pl'] * (100)
  print('Price to Earnings de: ', round(pl_inverso, 2), '%')
  titulo_public = 6.07 + 4.14   #Pegar tesouro IPCA a longo prazo + anual, ou SELIC atual
  print('Taxa do titulo_public: ', titulo_public, '%')

  bonds =  pl_inverso - titulo_public

  if bonds > titulo_public:
    return f"PL_Bonds: Taxa de retorno superior com: {bonds:.2f}%"
  else:
    return f"PL_Bonds: Taxa de retorno inferior com: {bonds:.2f}%"




print(pl_bonds(ticker))




Price to Earnings de:  6.76 %
Taxa do titulo_public:  10.21 %
PL_Bonds: Taxa de retorno inferior com: -3.45%


In [ ]:
#Função de lucro e dividendo do livro Valuation do Damodaran
"""
Ativo
Roe atual
Roe esperado
Tx crescimento esperado
LPA atual
Payout
capital = taxa de crescimento do Capital Social ou patrimonio liq. Ve no site quanto cresceu de um ano para outro.
"""
def valuation_damodaran(ativo, lpa, payout, anos):
    roe_atual = ativo['roe']
    capital = 0 / 100
    roe_esperado = roe_atual / (1 + capital)
    tx_cresc_esperado = (roe_esperado * (1 - payout / 100)) / 100

    tabela = [(0, lpa, lpa * (1 + tx_cresc_esperado), lpa * (payout / 100))]
    for ano in range(1, anos):
        lpa_futuro = tabela[-1][1] * (1 + tx_cresc_esperado)
        dy_futuro = lpa_futuro * (payout / 100)
        tabela.append((ano, lpa_futuro, lpa_futuro * (1 + tx_cresc_esperado), dy_futuro))

    return tabela

resultado = valuation_damodaran(ticker, calc_lpa, 62.0, 7)

print("Ano     LPA futuro   Dy por ação futuro")
for ano, lpa_futuro, _, dy_futuro in resultado:
    print(f"ano {ano} R$ {lpa_futuro:.2f}     R$ {dy_futuro:.2f}")






Ano     LPA futuro   Dy por ação futuro
ano 0 R$ 0.73     R$ 0.45
ano 1 R$ 0.73     R$ 0.45
ano 2 R$ 0.73     R$ 0.45
ano 3 R$ 0.73     R$ 0.45
ano 4 R$ 0.73     R$ 0.46
ano 5 R$ 0.73     R$ 0.46
ano 6 R$ 0.74     R$ 0.46


In [ ]:
##Aqui consigo usar o calc_payout, porem ta vindo errado, e se deixar dy_futuro = lpa_futuro * (payout / 100) o valor vem 0 em tudo...
##Mexer no cod ainda...

def valuation_damodaran(ativo, lpa, payout, anos):
    roe_atual = ativo['roe']
    capital = 11.61 / 100
    roe_esperado = roe_atual / (1 + capital)
    tx_cresc_esperado = (roe_esperado * (1 - payout / 100)) / 100

    tabela = [(0, lpa, lpa * (1 + tx_cresc_esperado), lpa * (payout / 100))]
    for ano in range(1, anos):
        lpa_futuro = tabela[-1][1] * (1 + tx_cresc_esperado)
        dy_futuro = lpa_futuro * (payout)
        tabela.append((ano, lpa_futuro, lpa_futuro * (1 + tx_cresc_esperado), dy_futuro))

    return tabela

resultado = valuation_damodaran(ticker, calc_lpa, calc_payout, 7)

print("Ano     LPA futuro   Dy por ação futuro")
for ano, lpa_futuro, _, dy_futuro in resultado:
    print(f"ano {ano} R$ {lpa_futuro:.2f}     R$ {dy_futuro:.2f}")





Ano     LPA futuro   Dy por ação futuro
ano 0 R$ 0.73     R$ 0.00
ano 1 R$ 0.73     R$ 0.46
ano 2 R$ 0.73     R$ 0.46
ano 3 R$ 0.74     R$ 0.46
ano 4 R$ 0.74     R$ 0.46
ano 5 R$ 0.74     R$ 0.46
ano 6 R$ 0.74     R$ 0.46


In [ ]:
filtro = df[(df.divbpatr < 1.5) & (df.dy > 0.06)].head(10)
filtro.shape

(10, 20)

In [ ]:
filtro

Multiples,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,evebitda,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y
papel,,,,,,,,,,,,,,,,,,,,
ABCB4,23.07,6.42,0.93,0.000,0.0667,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.1450,0.00,1.462000e+07,5.871730e+09,0.00,0.4586
ABEV3,12.01,13.05,2.40,2.373,0.0608,1.426,-42.60,11.12,-11.87,10.37,7.66,0.2133,0.1876,0.1790,0.1836,0.89,3.816040e+08,7.896930e+10,0.04,0.1211
AFLT3,7.49,18.44,1.81,7.878,0.0608,1.583,9.24,15.54,15.55,15.09,14.99,0.5070,0.4271,0.1080,0.0980,4.16,6.467140e+03,2.614890e+08,0.00,0.1827
AGRO3,25.30,9.86,1.31,2.141,0.1270,0.792,4.26,10.62,-12.37,14.01,10.28,0.2016,0.2172,0.0848,0.1334,2.25,7.522420e+06,1.976040e+09,0.53,0.2185
ALLD3,7.48,5.71,0.44,0.119,0.2717,0.206,0.61,3.90,1.42,3.95,2.90,0.0306,0.0209,0.0918,0.0766,2.02,1.178650e+06,1.599170e+09,0.34,0.1045
AURE3,11.48,-36.14,0.93,1.857,0.2962,0.396,5.50,12.57,-1.07,16.05,9.43,0.1477,-0.0514,0.0363,-0.0257,1.54,5.331190e+07,1.236900e+10,0.52,0.5348
BALM3,12.00,12.80,0.82,0.692,0.0739,0.471,1.03,5.11,1.62,3.41,2.81,0.1354,0.0526,0.1278,0.0637,2.82,1.441470e+03,1.442400e+08,0.14,0.0272
BALM4,10.49,11.19,0.71,0.605,0.0845,0.411,0.90,4.47,1.42,2.77,2.28,0.1354,0.0526,0.1278,0.0637,2.82,2.803000e+03,1.442400e+08,0.14,0.0272
BBAS3,27.55,4.70,0.96,0.000,0.0844,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.2054,0.00,4.767580e+08,1.636380e+11,0.00,0.2434


In [ ]:
#Decio Bazin com metodos a escolha
def metodo_bazin(dataframe):
    # Filtrando o DataFrame
    filtered_stocks = dataframe[(dataframe['divbpatr'] < 1.5) & (dataframe['dy'] > 0.06) & (dataframe['evebitda'] < 5) & (dataframe['pl'] < 10) & (dataframe['pl'] > 0) ]

    sorted_stocks = filtered_stocks.sort_values(by='dy', ascending=False)

    top_10_stocks = sorted_stocks.head(10)

    return top_10_stocks


top_dividend_stocks = metodo_bazin(df)
display(top_dividend_stocks)

Multiples,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,...,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,num_crit_atendidos
papel,,,,,,,,,,,,,,,,,,,,,
TRPN3,0.71,0.44,0.54,0.263,1.0474,0.399,0.59,0.36,0.59,-0.35,...,0.7228,0.5929,5.2301,1.2235,3.61,0.000000e+00,5.935800e+07,0.00,0.0577,4
ALLD3,7.48,5.71,0.44,0.119,0.2717,0.206,0.61,3.90,1.42,3.95,...,0.0306,0.0209,0.0918,0.0766,2.02,1.178650e+06,1.599170e+09,0.34,0.1045,4
CMIN3,4.98,7.65,2.55,1.447,0.1560,0.920,3.95,4.00,-4.55,3.76,...,0.3617,0.1890,0.3786,0.3336,2.14,5.069320e+07,1.069870e+10,0.76,0.1009,4
GOAU4,10.64,4.29,0.66,0.159,0.1449,0.146,0.60,1.21,2.83,1.78,...,0.1322,0.1101,0.1430,0.1528,2.61,8.791900e+07,1.676100e+10,0.65,0.1898,4
GOAU3,10.69,4.31,0.66,0.160,0.1442,0.147,0.61,1.21,2.84,1.79,...,0.1322,0.1101,0.1430,0.1528,2.61,7.551870e+05,1.676100e+10,0.65,0.1898,4
BMGB4,3.24,9.23,0.48,0.000,0.1357,0.000,0.00,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.0521,0.00,1.989930e+06,3.925170e+09,0.00,-0.0607,2
CSMG3,21.57,5.95,1.08,1.108,0.1285,0.578,15.42,4.26,-2.01,6.18,...,0.2602,0.1863,0.1509,0.1821,1.26,3.495750e+07,7.573830e+09,0.63,0.0907,4
SOND3,46.01,9.53,1.88,0.694,0.1204,1.149,3.44,4.19,5.42,3.24,...,0.1655,0.0729,0.3772,0.1972,2.26,0.000000e+00,6.019100e+07,0.00,0.3064,4
VALE3,62.74,7.36,1.52,1.382,0.1114,0.622,30.80,3.46,-1.62,4.11,...,0.3997,0.1900,0.2004,0.2068,1.12,1.601140e+09,1.872110e+11,0.39,0.0519,4


In [ ]:
#Decio Bazin Só com DY e pouca divida

def metodo_bazin(dataframe):
    filtered_stocks = dataframe[(dataframe['divbpatr'] < 1.5) & (dataframe['dy'] > 0.06)]

    sorted_stocks = filtered_stocks.sort_values(by='dy', ascending=False)

    top_10_stocks = sorted_stocks.head(10)

    return top_10_stocks


top_10_dividend_stocks = metodo_bazin(df)
display(top_10_dividend_stocks)


Multiples,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,evebitda,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y
papel,,,,,,,,,,,,,,,,,,,,
BRPR3,131.17,-0.57,0.05,0.695,12.0289,0.038,1.73,12.11,-0.49,-3.70,-2.55,0.0574,-1.2398,0.0035,-0.0934,1.12,0.0,1.136910e+09,0.07,-0.2505
TRPN3,0.71,0.44,0.54,0.263,1.0474,0.399,0.59,0.36,0.59,-0.35,-0.35,0.7228,0.5929,5.2301,1.2235,3.61,0.0,5.935800e+07,0.00,0.0577
CASH3,4.56,-21.37,0.51,1.214,0.5273,0.405,0.65,-5.58,0.73,3.76,5.21,-0.2176,-0.0782,-0.2296,-0.0239,5.63,14921100.0,7.764790e+08,0.00,0.4701
EPAR3,7.46,-7.13,1.38,0.242,0.5183,1.092,4.49,6.77,10.28,6.39,2.65,0.0358,-0.0340,0.1732,-0.1942,4.26,109601.0,8.007900e+07,0.01,0.0850
BPAT33,20.00,0.00,0.82,0.000,0.3572,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.00,0.0,8.764920e+08,0.00,0.0000
AURE3,11.48,-36.14,0.93,1.857,0.2962,0.396,5.50,12.57,-1.07,16.05,9.43,0.1477,-0.0514,0.0363,-0.0257,1.54,53311900.0,1.236900e+10,0.52,0.5348
ALLD3,7.48,5.71,0.44,0.119,0.2717,0.206,0.61,3.90,1.42,3.95,2.90,0.0306,0.0209,0.0918,0.0766,2.02,1178650.0,1.599170e+09,0.34,0.1045
GRND3,6.11,9.71,1.51,2.265,0.2270,1.353,2.38,14.62,2.41,12.59,10.08,0.1549,0.2333,0.1200,0.1551,7.02,12220000.0,3.659440e+09,0.03,0.0622
LEVE3,32.95,5.97,4.91,1.029,0.1840,1.594,7.28,5.50,-20.75,6.17,5.48,0.1871,0.1723,0.3876,0.8219,1.58,17747900.0,9.103790e+08,0.98,0.1789


In [ ]:
#Verificação de critérios para ver a quantidade de ações que atendem

# Verificar o critério divbpatr < 1.5
print("Número de ações com divbpatr < 1.5:", len(df[df['divbpatr'] < 1.5]))

# Verificar o critério dy > 0.06
print("Número de ações com dy > 0.06:", len(df[df['dy'] > 0.06]))

# Verificar o critério evebit / evebitda < 5
print("Número de ações com evebit / evebitda < 5:", len(df[df['evebit'] / df['evebitda'] < 5]))

# Verificar o critério roe > 10
print("Número de ações com roe > 10:", len(df[df['roe'] > 0.1 * 100]))

# Verificar o critério mrgliq > 10
print("Número de ações com mrgliq > 10:", len(df[df['mrgliq'] > 10]))

# Verificar o critério liqc > 1
print("Número de ações com liqc > 1:", len(df[df['liqc'] > 1]))

# Verificar o critério pl < 10 e pl > 3
print("Número de ações com pl < 10 e pl > 3:", len(df[(df['pl'] < 10) & (df['psr'] > 3)]))


Número de ações com divbpatr < 1.5: 799
Número de ações com dy > 0.06: 114
Número de ações com evebit / evebitda < 5: 834
Número de ações com roe > 10: 4
Número de ações com mrgliq > 10: 21
Número de ações com liqc > 1: 582
Número de ações com pl < 10 e pl > 3: 77


In [ ]:
# Verificar os critérios e calcular o número de critérios atendidos
df['num_crit_atendidos'] = (
    (df['divbpatr'] < 1.5).astype(int) +
    (df['dy'] > 0.06).astype(int) +
    ((df['evebit'] / df['evebitda']) < 5).astype(int) +
    (df['roe'] > 0.1 * 100).astype(int) +
    (df['mrgliq'] > 10).astype(int) +
    (df['liqc'] > 1).astype(int) +
    ((df['pl'] < 10) & (df['psr'] > 3)).astype(int)
)

max_crit_atendidos = df['num_crit_atendidos'].max()

acoes_max_combinacoes = df[df['num_crit_atendidos'] == max_crit_atendidos].sort_values(by='dy', ascending=False)

num_acoes_max_combinacoes = len(acoes_max_combinacoes)

print("Número de ações que atendem ao critério com o maior número de combinações:", num_acoes_max_combinacoes)
display(acoes_max_combinacoes)



Número de ações que atendem ao critério com o maior número de combinações: 10


Multiples,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,...,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,num_crit_atendidos
papel,,,,,,,,,,,,,,,,,,,,,
CEBR6,23.96,9.41,1.71,4.899,0.1217,1.220,2.57,13.64,2.70,8.25,...,0.3592,0.6666,0.1778,0.1820,6.97,170717.0,1.008340e+09,0.00,-0.3212,5
CEBR5,22.99,9.03,1.64,4.700,0.1153,1.170,2.46,13.09,2.59,7.69,...,0.3592,0.6666,0.1778,0.1820,6.97,128489.0,1.008340e+09,0.00,-0.3212,5
ITSA3,9.67,7.42,1.20,13.528,0.0884,0.948,12.71,132.63,-17.69,136.72,...,0.1020,1.8933,0.0078,0.1623,2.64,1752060.0,8.295200e+10,0.13,0.1211,5
ITSA4,9.69,7.43,1.21,13.555,0.0882,0.950,12.74,132.91,-17.73,137.00,...,0.1020,1.8933,0.0078,0.1623,2.64,176903000.0,8.295200e+10,0.13,0.1211,5
TRPL3,32.72,7.59,1.24,3.469,0.0674,0.600,6.10,6.24,-1.78,8.41,...,0.5558,0.4653,0.1017,0.1635,2.45,74418.3,1.738010e+10,0.53,0.1796,5
TAEE3,11.70,8.84,1.81,3.597,0.0638,0.622,6.90,6.34,-1.30,10.74,...,0.5674,0.4068,0.1061,0.2048,2.00,1036470.0,6.679310e+09,1.45,0.0993,5
TAEE11,35.25,8.88,1.82,3.612,0.0636,0.624,6.93,6.37,-1.31,10.77,...,0.5674,0.4068,0.1061,0.2048,2.00,65390100.0,6.679310e+09,1.45,0.0993,5
BRGE6,12.92,32.46,0.72,267.653,0.0633,0.318,0.83,-1.36,1.62,0.92,...,-196.3690,13.8129,-0.5003,0.0221,3.06,5254.0,1.324180e+09,0.00,-0.4561,5
TAEE4,11.87,8.97,1.84,3.649,0.0629,0.631,7.00,6.43,-1.32,10.83,...,0.5674,0.4068,0.1061,0.2048,2.00,2266210.0,6.679310e+09,1.45,0.0993,5
